#Deep MNIST para Expertos

TensorFlow es una poderosa libreria para hacer calculos numericos a gran escala. Una de las tareas que realiza esta libreria es implemntar y entrenar deep neural networks. En este tutorial se aprende los bloques basicos para construir un modelo TensorFlow mientras se construye un clasificador deep convolutional MNIST.

#Preparacion

Antes de crear el modelo primero se debe cargar el dataset MNIST y despues iniciar una sesion TensorFlow.

#Cargar la informacion de MNIST

Para cargar la informacion se incluye un script que automaticamente descarga e importa el dataset de MNIST. Cuando se ejecuta este script se crea un directorio con en nombre de 'MNIST_data' en el cual se guardan los archivos con los datos.

a continuacion se presentan las lineas que ejecutan dicho script:

In [1]:
import input_data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


La variable mnist contiene los arreglos de tipo numpy de entrenamiento, validacion y testing. Ademas provee una funcion que permite iterar atraves de los minibatches de datos los cuales se utilizan mas adelante.

#Comenzar una sesion interactiva en TensorFlow

TensorFlow se basa en un eficiente backend de C++ para hacer los calculos. La conexion para este backend se llama sesion. El uso mas comun para programas TensorFlow es primero crear un grafo y despues iniciar una sesion.

Aqui se utiliza por conveniciencia la clase InteractiveSession la cual hace que TensorFlow sea aun mas flexible acerca de la estructura del codigo Tambien permite intercalar operaciones las cuales construyen un grafo de calculos con el cual se ejecutara el grafo. Esto es particularmente conveniente cuando se trabaja con contextos interactivos como iPython. 

A continuacion se inicia la sesion:

In [2]:
import tensorflow as tf
sess = tf.InteractiveSession()

#Computacion Grafica (Cambiar este nombre!) grafo computacional ..

Para realizar calculos numericos eficientes en python usualmente se utilizan librerias como NumPy que hacen operaciones costosas como las multiplicacion de matrices fuera de Python utilizando codigo eficiente implementado en otro lenguaje. Desafortunadamente existe una gran cantidad de overhead al cambiar de una operacion a otra.

TensorFlow utiliza librerias implementadas en otros lenguajes pero evitando el overhead. Provee un grafo con operaciones interactivas las cuales corren fuera de python.

El rol de codigo en python es construir un grafo externo y dictarle que partes de el grafo se van a utilizar.

# Construir el Modelo de la Regresion Softmax

En esta seccion se construye el modelo de la regresion softmax con una sola capa. En la siguiente seccion se extiene el caso de la regresion softmax con multicapas en una convolutional network.

#Placeholders

Para comenzar a construir un grafo computacional se crean nodos para las imagenes de entrada y salidas para las clases.

In [3]:
x = tf.placeholder("float", shape=[None, 784])
y_ = tf.placeholder("float", shape=[None, 10])

Aqui x e y_ no tienen valores especificos ya que son placeholders esto significa que se les asignara un valor de entrada cuando se le pida a TensorFlow ejecutar alguna operacion.

Las imagenes de entrada x consisten de un tensor 2d de numeros de punto flotante. Aqui se les asigna un tamano de [None, 784], en donde 784 es la dimension de una imagen de MNIST y None indica que la primera dimension corresponde al tamano del batch este puede tener cualquier tamano. La salida de las clases es -_ el cual consiste de un tensor 2d donde cada renglon es un vector "one-hot 10-dimensional" indicando cual es el digito de la clase a la cual corresponde la imagen a tratar.

#Variables

Ahora se definen los pesos W y los bias B para el modelo. Se pueden pensar que son entradas adicionales, pero TensorFlow tiene una manera de manejar este tipo de entradas como variables. Una variable es un valor que vive en el grafo computacional de TensorFlow . Estas pueden ser usadas e incluso modificadas por calculos. Comunmente en aplicaciones de machine learning los modelos utilizan parametros variables.

A continuacion se declaran dichas bariables para el peso y el bias:

In [4]:
W = tf.Variable(tf.zeros([784,10]))
b = tf.Variable(tf.zeros([10]))

Primero se pasa el valor inicial para cada parametro en la llamada a la funcion tf.variable. En este caso se inicializan w y b con tensores llenos de 0's. W es una matriz de 784 x 10 (Porque se tienen 784 entradas y 10 salidas) y b es un vector de dimension 10 (Porque se tienen 10 clases).

Antes de que las variables se utilicen dentro de una sesion se deben inicializar usando esa sesion. Este paso toma los valores iniciales (En este caso los tensores llenos de 0's ) e inicializa todas las variables a la vez

A continuacion se inicializan las variables:

In [5]:
sess.run(tf.initialize_all_variables())

#Prediciendo la clase y la funcion de costo

Ahora se puede implementar el modelo de regresion. Solo toma una linea de codigo! Se multiplica el vector de imagenes de entrada x por la matriz de pesos W y se agrega el bias b con esto se ejecutan las probabilidades de softmax asignandola a cada clase.

In [6]:
y = tf.nn.softmax(tf.matmul(x,W) + b)

La funcion de costo se minimiza durante el entrenamiento. La funcion de costo a utilizar es cross-entropy  la cual se utiliza la salida conocida y la de la prediccion del modelo.

In [7]:
cross_entropy = -tf.reduce_sum(y_*tf.log(y))